In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

df =pd.read_csv("drive/MyDrive/CMPE-257-Project/complaints.csv")
df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (9,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2022-11-22,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,NaN,"EQUIFAX, INC.",OK,74447.0,Servicemember,NaN,Web,2022-11-22,In progress,Yes,NaN,6231498
1,2022-11-16,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,NaN,Experian Information Solutions Inc.,NJ,7093.0,NaN,NaN,Web,2022-11-16,In progress,Yes,NaN,6205383
2,2022-11-09,Debt collection,Credit card debt,Took or threatened to take negative or legal a...,Threatened or suggested your credit would be d...,NaN,NaN,"Genesis FS Card Services, Inc.",TX,75038.0,NaN,NaN,Web,2022-11-09,Closed with explanation,Yes,NaN,6182699
3,2022-11-07,Mortgage,Other type of mortgage,Trouble during payment process,NaN,NaN,NaN,NORTHERN OHIO INVESTMENT COMPANY,MO,63031.0,NaN,Other,Web,2022-11-08,Untimely response,No,NaN,6173945
4,2022-11-07,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,NaN,NaN,"R & R Collection Service, Inc.",MO,64154.0,NaN,NaN,Phone,2022-11-07,Untimely response,No,NaN,6175998


In [4]:
pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 8.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=76feac993e508e59bdff084b9934ed79d51a556c89537fe50edc4651fa482c3f
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=3ea7167c1eac609fcf01674c0bbc787093547db07257d79b5d6c86142d507edb
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyLDAvis sklearn


In [6]:
import numpy as np

import re
import string

import spacy

import gensim
from gensim import corpora

import pyLDAvis
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:

def text_clean(text): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))]) 
    
    return text2.lower()

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
df = df.rename(columns={'Product' : "Product",
                       'Consumer complaint narrative' : "Complaint"})

In [13]:
df = df[pd.notnull(df['Complaint'])]
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Complaint,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
38,2022-11-09,Debt collection,Other debt,Attempts to collect debt not owed,Debt was paid,XXXX. I do not owe any money to XXXX XXXX. I ...,Company has responded to the consumer and the ...,"I.C. System, Inc.",CA,90807.0,NaN,Consent provided,Web,2022-11-09,Closed with explanation,Yes,NaN,6185206
39,2022-11-09,Debt collection,Other debt,False statements or representation,Indicated you were committing crime by not pay...,XXXX is attempting to collect funds for Valuat...,Company believes it acted appropriately as aut...,Credit International Corporation,GA,30096.0,Servicemember,Consent provided,Web,2022-11-09,Closed with explanation,Yes,NaN,6179541
41,2022-11-07,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Credit inquiries on your report that you don't...,EXPERIAN I didnt consent to these Inquiries Al...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,GA,30813.0,Servicemember,Consent provided,Web,2022-11-07,Closed with explanation,Yes,NaN,6176367
68,2022-11-11,Checking or savings account,Savings account,Managing an account,Deposits and withdrawals,Citibank froze my account that contained {$200...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",NY,12065.0,NaN,Consent provided,Web,2022-11-11,Closed with explanation,Yes,NaN,6189622
129,2022-10-13,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,In accordance with the fair credit reporting a...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",GA,30043.0,NaN,Consent provided,Web,2022-10-13,Closed with explanation,Yes,NaN,6079679


In [14]:
sample_df = df.sample(100000, random_state=42)
complaint_text = sample_df['Complaint']

In [15]:
complaint_text = complaint_text.apply(text_clean)

In [16]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

In [17]:
complaint_text = complaint_text.apply(remove_stopwords)

In [19]:
pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
!python -m spacy download en_core_web_md 

2022-12-06 08:35:22.127372: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.8 MB 1.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [22]:
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

In [23]:
text_list = complaint_text.tolist()
tokenized_complaints = lemmatization(text_list)
print(tokenized_complaints[1])

['vehicle', 'company', 'lien', 'title', 'company', 'business', 'assistance', 'thank']


In [24]:
dictionary = corpora.Dictionary(tokenized_complaints)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_complaints]

In [ ]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=10, random_state=100,
                chunksize=1000, passes=50,iterations=100)

In [26]:
lda_model.print_topics()

[(0,
  '0.124*"card" + 0.075*"credit" + 0.051*"charge" + 0.028*"account" + 0.017*"purchase" + 0.016*"dispute" + 0.014*"capital" + 0.012*"balance" + 0.012*"transaction" + 0.012*"xxxx"'),
 (1,
  '0.122*"account" + 0.064*"bank" + 0.056*"money" + 0.037*"check" + 0.028*"fund" + 0.017*"transaction" + 0.015*"deposit" + 0.014*"day" + 0.014*"transfer" + 0.011*"claim"'),
 (2,
  '0.117*"debt" + 0.067*"collection" + 0.031*"company" + 0.017*"claim" + 0.016*"original" + 0.016*"contract" + 0.015*"agency" + 0.014*"letter" + 0.013*"validation" + 0.012*"legal"'),
 (3,
  '0.151*"payment" + 0.034*"month" + 0.033*"late" + 0.031*"amount" + 0.028*"balance" + 0.023*"time" + 0.021*"account" + 0.018*"interest" + 0.018*"statement" + 0.018*"xxxx"'),
 (4,
  '0.040*"call" + 0.037*"phone" + 0.034*"number" + 0.030*"time" + 0.026*"email" + 0.019*"information" + 0.019*"service" + 0.018*"company" + 0.016*"letter" + 0.015*"address"'),
 (5,
  '0.132*"credit" + 0.083*"report" + 0.047*"inquiry" + 0.038*"identity" + 0.032*"t

In [28]:
import pyLDAvis.gensim_models

In [30]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.122563  0.049297       1        1  15.727243
4      0.157846 -0.065951       2        1  13.569094
8      0.215588 -0.122227       3        1  12.439348
3      0.176858  0.118418       4        1  12.426585
2      0.025507 -0.300660       5        1   9.673373
1      0.130756  0.107296       6        1   8.702724
5     -0.132832  0.089274       7        1   8.359546
0      0.103985  0.184723       8        1   7.809557
7     -0.224592 -0.110798       9        1   7.248086
9     -0.330552  0.050629      10        1   4.044444, topic_info=        Term           Freq          Total Category  logprob  loglift
195  payment   82261.000000   82261.000000  Default  30.0000  30.0000
61    credit  169893.000000  169893.000000  Default  29.0000  29.0000
1    account  149150.000000  149150.000000  Default  28.0000  28.0000
68      debt   50458.000000   50458.000000  Default  27.0000  27.0000
159     loan   51387.000000   51387.000000  Default  26.0000  26.0000
..       ...            ...            ...      ...      ...      ...
368     item     864.560735   21059.433067  Topic10  -5.3127   0.0149
234   report     935.191941   83612.588065  Topic10  -5.2341  -1.2854
652     open     732.899114    7075.108490  Topic10  -5.4779   0.9405
186   number     691.476156   31135.876330  Topic10  -5.5361  -0.5995
452     late     653.094114   18179.920590  Topic10  -5.5932  -0.1185

[675 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
3059     10  0.993347  15usc
753      10  0.998057  1666b
467      10  0.997784  1681b
468      10  0.996071  1681n
423       1  0.036074   able
...     ...       ...    ...
312       5  0.010002   year
312       6  0.016924   year
312       7  0.071416   year
312       8  0.051331   year
312      10  0.015443   year

[1222 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 9, 4, 3, 2, 6, 1, 8, 10])